In [ ]:
import xml.dom.minidom as minidom
import xml.etree.ElementTree as xmlET
from future.utils import iteritems

from pyPRMS.prms_helpers import read_xml

In [ ]:
parent_modules = {'potet': ['potet_hamon', 'potet_hs', 'potet_jh', 'potet_pan', 'potet_pm', 'potet_pm_sta', 'potet_pt', 'potet_hru'],
                  'solar_radiation': ['ccsolrad', 'ddsolrad', 'solar_radiation_hru'], #??soltab??
                  'streamflow': ['muskingum', 'muskingum_lake', 'strmflow_in_out'], 
                  'temperature': ['temp_1sta', 'temp_laps', 'temp_dist2', 'temperature_hru'],
                  'precipitation': ['precip_1sta', 'precip_laps', 'precip_dist2', 'precipitation_hru'],
                  'transpiration': ['transp_frost', 'transp_tindex']}

In [ ]:
# Read in the parameters.xml file
global_params_file = '<path>/parameters.xml'
params_root = read_xml(global_params_file)

module_dict = {}
dims_by_param = {}

# Populate parameterSet with all available parameter names
for param in params_root.findall('parameter'):
    xml_param_name = param.attrib.get('name')

    # Build dictionary of dimensions (in correct order) for each parameter
    for cdim in param.findall('./dimensions/dimension'):
        dim_name = cdim.attrib.get('name')
        dim_pos = int(cdim.find('position').text)
        dims_by_param.setdefault(xml_param_name, []).insert(dim_pos, dim_name)    
    
    mods_list = []
    for cmod in param.findall('./modules/module'):
        mods_list.append(cmod.text)
        
    in_parent = False
    
    # If a parameter is required by all the child modules of a parent then add
    # only the parent module to the dictionary.
    for pp, tt in iteritems(parent_modules):
        if len(set(tt).difference(set(mods_list))) == 0:
            module_dict.setdefault(pp, []).append(xml_param_name)
            in_parent = True
            
            # There could be other child modules remaining so add them too
            for mm in set(tt).symmetric_difference(set(mods_list)):
                module_dict.setdefault(mm, []).append(xml_param_name)
                
            break
    
    if not in_parent:
        for mm in mods_list:
            module_dict.setdefault(mm, []).append(xml_param_name)

In [ ]:
# List the parameters for each module
for xx, yy in iteritems(module_dict):
    print(xx)
    for zz in yy:
        print('\t{}'.format(zz))

In [ ]:
# List dimensions for each parameter
for xx, yy in iteritems(dims_by_param):
    print(xx, yy)

### Write parameters into csv format

In [ ]:
import pyPRMS.NhmParamDb_v2 as nhm
import pandas as pd

In [ ]:
workdir = '<path>/paramdb_v2'

pdb = nhm.NhmParamDb_v2(workdir)

In [ ]:
df = pdb.parameters.get('dday_slope').as_dataframe

In [ ]:
pdb.parameters.get('dday_slope').dimensions.tostructure()

In [ ]:
# Create a position-ordered list of dimension names for the parameter
dim_list = []

for dm, dd in iteritems(pdb.parameters.get('dday_slope').dimensions.tostructure()):
    dim_list.insert(dd['position'], dm)

print(','.join(dim_list))

In [ ]:
# Write a CSV file of parameter data with the dimension names as a first-line comment

fhdl = open('dday_slope.csv', 'w')

# Write the dimension info as first line
fhdl.write('# {}\n'.format(','.join(dim_list)))

# Write the data to a csv file
df.index += 1
df.to_csv(fhdl, header=False, index=True)

fhdl.close()